<a href="https://colab.research.google.com/github/michal-g/Notebooks-to-Packages/blob/main/predicting-ufo-sightings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start by scraping the datasets from the sightings reports website. The reports portal contains a link to a table for each historical month, so we have to go through the links and parse each table individually. For now, we will only consider sightings from the 1990s to speed up this step.

Note that at this stage it is easier to store the sightings as a list of dictionaries that can be progressively grown as we scan through the website for sighting records.

In [ ]:
import itertools
import re
import requests
from bs4 import BeautifulSoup


# initialize assets for scraping the reports portal
base_url = 'https://nuforc.org/webreports'
grab = requests.get('/'.join([base_url, 'ndxevent.html']))

# initialize data structures for storing parsed data
sightings = []
col_labels = ['Date', 'City', 'State', 'Country', 'Shape', 'Duration',
              'Summary', 'Posted', 'Images']

# for each link to a month's data, create assets for scraping that table
for month_link in BeautifulSoup(grab.text, 'html.parser')(
    'a', string=re.compile("[0-9]{2}\/199[0-9]")):
  month_grab = requests.get('/'.join([base_url, month_link.get('href')]))

  # the HTML formatting is kind of weird; we first grab the outermost of a
  # recursively defined set of table elements
  table_data = BeautifulSoup(month_grab.text, 'html.parser')('tr')[1]('td')
  cur_sighting = None

  # then we loop over a set of table entries that are defined as one big row???
  # maybe there's an easier way to do this but that's ok
  for lbl, col in zip(itertools.cycle(col_labels), table_data):
    if lbl == 'Date':
      if cur_sighting is not None:
        sightings.append(cur_sighting)

      # start a new sighting record, after adding the last record to the list of
      # sightings if this is not the first row
      cur_sighting = {'Date': col.string}

    else:
      cur_sighting[lbl] = col.string

  # accounting for the last row
  if cur_sighting is not None:
    sightings.append(cur_sighting)


Once we've created a list containing all the sightings, we can convert it into a tabular format that is more convenient for us. We will remove any sightings from outside the US to simplify further analyses.

In [ ]:
import pandas as pd


# we will be very careful about filtering sightings that can be mapped to states
valid_states = {
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI',
    'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN',
    'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH',
    'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
    }

# create a table for the sightings data and only consider valid unique sightings
sights_df = pd.DataFrame(sightings).drop_duplicates()
sights_df = sights_df.loc[(sights_df.Country == 'USA')
                          & sights_df.State.isin(valid_states), :]

# parse the date information into a more useful format
sights_df['Date'] = pd.to_datetime([dt.split()[0] for dt in sights_df['Date']],
                                   format='%m/%d/%y')

print(sights_df)


Let's examine the total number of sightings from across the year for each state. We see that California is a very popular destination!

In [ ]:
import plotly.express as px


state_totals = sights_df.groupby('State').size()

fig = px.choropleth(locations=[str(x) for x in state_totals.index],
                    scope="usa", locationmode="USA-states",
                    color=state_totals.values,
                    range_color=[0, state_totals.max()],
                    color_continuous_scale=['white', 'red'])
fig.show()


To better understand how sightings vary across the year, we'll animate the weekly numbers of sightings for each state.

In [ ]:
import imageio
from IPython.display import Image
from pathlib import Path


# create a Week x State table containing total weekly sightings for each state;
# note that we have to take into account "missing" weeks that did not have any
# sightings in any states
state_table = sights_df.groupby(['Date', 'State']).size().unstack().fillna(0)
state_table = state_table.reindex(
  index=pd.date_range('01-01-1990', '12-31-1999'), fill_value=0).sort_index()
state_weeklies = state_table.groupby(
  pd.Grouper(axis=0, freq='W', sort=True)).sum()

# create a list of individual files and a place to save them
plt_files = list()
!mkdir -p map-plots

# create a map of sightings by state for each week
for week, week_counts in state_weeklies.iterrows():
    day_lbl = week.strftime('%F')
    state_locs = [str(x) for x in week_counts.index.get_level_values('State')]

    fig = px.choropleth(locations=state_locs, locationmode="USA-states",
                        title=day_lbl, scope='usa',
                        color=week_counts.values, range_color=[0, 10],
                        color_continuous_scale=['white', 'black'])

    # save the map to file and keep track of the file name
    plt_file = Path("map-plots", f"counts_{day_lbl}.png")
    fig.write_image(plt_file, format='png')
    plt_files += [imageio.v2.imread(plt_file)]

# create an animation using the individual weekly maps
imageio.mimsave(Path("map-plots", "counts.gif"), plt_files, duration=0.03)
Image(filename=str(Path("map-plots", "counts.gif")))


A time series regression model can predict the number of sightings that will take place on each of a series of weeks. We set up a cross-validation process in which the model makes predictions across chunks of our year range by training on preceding chunks. The quality of the predictions is judged using root-mean-squared-error, which is useful as it places the error in the same scale as the original data.

To improve our model's performance, we can try to swap out or rearrange elements of the prediction pipeline, as well as tune the hyper-parameters of the input transformers and prediction algorithms. This is left as an exercise to the reader!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 9)
import numpy as np

from skits.preprocessing import ReversibleImputer
from skits.pipeline import ForecasterPipeline
from skits.feature_extraction import (AutoregressiveTransformer,
                                      SeasonalTransformer)

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit


pipeline = ForecasterPipeline([
    ('pre_scaler', StandardScaler()),
    ('features', FeatureUnion([
      ('ar_features', AutoregressiveTransformer(num_lags=52)),
      ('seasonal_features', SeasonalTransformer(seasonal_period=52)),
      ])),
    ('post_feature_imputer', ReversibleImputer()),
    ('post_feature_scaler', StandardScaler()),
    ('regressor', LinearRegression())
    ])

tscv = TimeSeriesSplit(n_splits=4)
cali_weeklies = state_weeklies.CA
cali_dates = cali_weeklies.index.values.reshape(-1, 1)
cali_values = cali_weeklies.values

real_values = list()
pred_values = list()

for train_index, test_index in tscv.split(cali_weeklies):
    pipeline.fit(cali_dates[train_index], cali_values[train_index])
    preds = pipeline.predict(cali_dates[test_index], to_scale=True)

    real_values += cali_values[test_index].flatten().tolist()
    pred_values += preds.flatten().tolist()

    plt.plot(cali_dates[test_index], cali_values[test_index], color='black')
    plt.plot(cali_dates[test_index], preds, color='red')

rmse_val = ((np.array(real_values) - np.array(pred_values)) ** 2).mean() ** 0.5
print(f"RMSE: {format(rmse_val, '.3f')}")
